In [ ]:
pip install opencv-python numpy matplotlib scikit-image tensorflow fiftyone kaggle


In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.metrics import silhouette_score
import time


In [ ]:
def get_dominant_color_efficientnet(image_path, k=3):
    """
    Identify the dominant color in an image using EfficientNet for feature extraction and k-means clustering.

    Args:
        image_path (str): Path to the input image.
        k (int): Number of clusters for k-means. Default is 3.

    Returns:
        dominant_color_rgb (tuple): Dominant color in RGB format.
        cluster_labels (numpy.ndarray): Cluster labels for each pixel.
        features_flat (numpy.ndarray): Feature vectors for clustering.
        processing_time (float): Time taken to process the image.
    """
    start_time = time.time()

    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image at path {image_path} could not be loaded.")

    # Resize the image to the input size required by EfficientNet (224x224)
    image_resized = cv2.resize(image, (224, 224))

    # Preprocess the image for EfficientNet
    image_preprocessed = preprocess_input(image_resized)

    # Load the pre-trained EfficientNetB0 model (without the top classification layer)
    model = EfficientNetB0(weights="imagenet", include_top=False)

    # Extract features for each pixel (use the last convolutional layer)
    features = model.predict(np.expand_dims(image_preprocessed, axis=0))

    # Reshape the features to (height * width, num_features)
    height, width, num_features = features.shape[1], features.shape[2], features.shape[3]
    features_flat = features.reshape((-1, num_features))

    # Apply k-means clustering to find dominant colors
    kmeans = KMeans(n_clusters=k, n_init=10)
    cluster_labels = kmeans.fit_predict(features_flat)

    # Get the cluster centroids (feature vectors of dominant colors)
    cluster_centers = kmeans.cluster_centers_

    # Map the clusters back to the original image pixels
    dominant_cluster = np.argmax(np.bincount(cluster_labels))  # Find the cluster with the most pixels

    # Extract the dominant color from the original image
    dominant_color_bgr = image_resized[cluster_labels == dominant_cluster].mean(axis=0).astype(int)
    dominant_color_rgb = tuple(reversed(dominant_color_bgr))  # Convert BGR to RGB

    processing_time = time.time() - start_time

    return dominant_color_rgb, cluster_labels, features_flat, processing_time

In [13]:

def visualize_results(image_path, dominant_color_rgb, cluster_labels):
    """
    Visualize the original image, dominant color, and color distribution.

    Args:
        image_path (str): Path to the input image.
        dominant_color_rgb (tuple): Dominant color in RGB format.
        cluster_labels (numpy.ndarray): Cluster labels for each pixel.
    """
    # Load the original image
    image = cv2.imread(image_path)
    

    # Create a figure for visualization
    plt.figure(figsize=(15, 5))

    # Plot the original image
    plt.subplot(1, 3, 1)
    plt.imshow(image)
    plt.title("Original Image")
    plt.axis("off")

    # Plot the dominant color
    plt.subplot(1, 3, 2)
    dominant_color_block = np.zeros((100, 100, 3), dtype=np.uint8)
    dominant_color_block[:, :] = dominant_color_rgb
    plt.imshow(dominant_color_block)
    plt.title(f"Dominant Color (RGB: {dominant_color_rgb})")
    plt.axis("off")

    # Plot the color distribution (cluster labels)
    plt.subplot(1, 3, 3)
    plt.hist(cluster_labels, bins=range(len(np.unique(cluster_labels)) + 1), edgecolor="black")
    plt.title("Color Distribution (Clusters)")
    plt.xlabel("Cluster")
    plt.ylabel("Number of Pixels")

    plt.tight_layout()
    plt.show()

In [14]:
def analyze_performance(features_flat, cluster_labels, processing_time):
    """
    Analyze the performance of the clustering process.

    Args:
        features_flat (numpy.ndarray): Feature vectors for clustering.
        cluster_labels (numpy.ndarray): Cluster labels for each pixel.
        processing_time (float): Time taken to process the image.
    """
    # Calculate silhouette score (clustering quality)
    silhouette_avg = silhouette_score(features_flat, cluster_labels)
    print(f"Silhouette Score: {silhouette_avg:.2f}")

    # Print processing time
    print(f"Processing Time: {processing_time:.2f} seconds")

In [15]:

def train_model(input_folder, k=3):
    """
    Process all images in the input folder to identify dominant colors and analyze performance.

    Args:
        input_folder (str): Path to the folder containing input images.
        k (int): Number of clusters for k-means. Default is 3.

    Returns:
        results (list): A list of dictionaries containing results for each image.
    """
    results = []

    # Loop through all images in the folder
    for filename in os.listdir(input_folder):
        if filename.lower().endswith((".jpg", ".png", ".jpeg")):  # Check for image files
            # Construct the full path to the image
            image_path = os.path.join(input_folder, filename)

            # Get the dominant color and other details

            dominant_rgb, cluster_labels, features_flat, processing_time = get_dominant_color_efficientnet(image_path, k=3)

            # Visualize the results
            visualize_results(image_path, dominant_rgb, cluster_labels)

            # Analyze performance
            analyze_performance(features_flat, cluster_labels, processing_time)

            # Save the results for this image
            results.append({
                "filename": filename,
                "dominant_color_rgb": dominant_rgb,
                "processing_time": processing_time,
                "silhouette_score": silhouette_score(features_flat, cluster_labels)
            })

            return results




In [16]:
input_folder = "/home/sala/iit/dsgp/preprocessed_data/general_data/" 
results = train_model(input_folder, k=3)

# Print the results
for result in results:
    print(f"Image: {result['filename']}")
    print(f"Dominant Color (RGB): {result['dominant_color_rgb']}")
    print(f"Processing Time: {result['processing_time']:.2f} seconds")
    print(f"Silhouette Score: {result['silhouette_score']:.2f}")
    print("-" * 40)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step


IndexError: boolean index did not match indexed array along axis 0; size of axis is 224 but size of corresponding boolean axis is 49

<!--  -->